In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import joblib
import numpy as np
import pandas as pd

# Modell és skálázó betöltése
try:
    model = joblib.load("random_forest_model.pkl")
    scaler = joblib.load("scaler.pkl")
except Exception as e:
    model = None
    scaler = None
    print("Nem sikerült betölteni a modellt vagy a skálázót:")
    print(e)

# Versenyzők és jellemzőik
driver_names = [
    "Hamilton", "Alonso", "Hülkenberg", "Pérez", "Ricciardo", "Bottas",
    "Magnussen", "Verstappen", "Sainz", "Ocon", "Stroll", "Gasly", "Leclerc",
    "Norris", "Russell", "Albon", "Tsunoda", "Zhou", "Piastri", "Sargeant",
    "Lawson", "Bearman", "Colapinto", "Doohan"
]

driver_features = [
    [277.8, 19.3, 4.510227272727272, 2.934496037282644, 5576.3],
    [122.9, 14.8, 6.668181818181818, 3.025237458351052, 2407.5],
    [4.6, 0.6, 10.716666666666667, 2.3088332933273357, 394.0],
    [256.0, 17.8, 5.2281818181818185, 3.865689952387723, 8115.9],
    [48.4, 6.5, 10.727158135981664, 3.308427682734087, 1481.6],
    [86.2, 8.7, 9.575668449197861, 3.7332074238839112, 2299.5],
    [8.8, 3.1, 10.358021390374333, 3.1318918307600323, 255.1],
    [466.7, 19.8, 2.364973262032086, 2.4645375755656693, 8238.9],
    [197.2, 17.2, 6.0, 2.9389624857580694, 4601.3],
    [71.8, 13.4, 8.659893048128342, 2.775851816121701, 1536.5],
    [49.3, 10.0, 9.79034090909091, 3.029448726678561, 2050.4],
    [61.2, 10.2, 9.759893048128342, 3.253463853199407, 1063.4],
    [216.4, 16.7, 5.4917112299465245, 2.786619093023865, 4556.8],
    [160.3, 16.8, 7.445989304812834, 4.051000636051318, 2453.2],
    [156.3, 13.8, 9.011764705882353, 3.758957021572792, 4244.4],
    [22.5, 4.9, 8.55179526355997, 2.341449943062, 430.6],
    [16.8, 5.0, 10.568181818181818, 3.1338059590382903, 507.1],
    [4.2, 2.1, 8.604545454545455, 2.0837944894022997, 368.8],
    [38.8, 4.4, 3.7636363636363637, 1.7134052573041718, 996.4],
    [0.4, 0.4, 5.363636363636363, 1.5936019263188346, 117.2],
    [0.8, 0.4, 4.88, 1.2132600710482482, 53.6],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0]
]

feature_names = ['total_points', 'points_count', 'avg_position', 'position_std', 'team_points']
feature_inputs = [widgets.FloatText(description=name) for name in feature_names]

# Widgetek
driver_dropdown = widgets.Dropdown(
    options=sorted(driver_names),
    value="Verstappen",
    description='Driver:'
)

predict_btn = widgets.Button(description='Predict', button_style='success')
output = widgets.Output()

# Versenyző adatainak betöltése
def set_driver_features(driver_name):
    idx = driver_names.index(driver_name)
    for i, val in enumerate(driver_features[idx]):
        feature_inputs[i].value = val

# Eseményfigyelő: versenyzőváltásra frissíti a mezőket
def update_feature_inputs(change):
    set_driver_features(change['new'])

driver_dropdown.observe(update_feature_inputs, names='value')

# Predikció logika
def predict_points(event):
    output.clear_output()
    if model is None or scaler is None:
        with output:
            print("A modell vagy a scaler nem elérhető.")
        return
    try:
        # DataFrame-ként alakítjuk át a bemenetet a feature nevek miatt
        features = np.array([f.value for f in feature_inputs]).reshape(1, -1)
        features_df = pd.DataFrame(features, columns=feature_names)

        # Skálázás + predikció
        scaled_features = scaler.transform(features_df)
        pred = model.predict(scaled_features)

        with output:
            print(f"Prediction for {driver_dropdown.value}:")
            print(f"Points: {pred[0]:.2f}")
    except Exception as e:
        with output:
            print("Hiba történt a predikció során:")
            print(str(e))

predict_btn.on_click(predict_points)

# Alapértelmezett versenyző beállítása
set_driver_features(driver_dropdown.value)

# Megjelenítés
display(widgets.VBox([driver_dropdown] + feature_inputs + [predict_btn, output]))
